In [ ]:
import importlib
import models
import helper_funtions
import preprocess
import settings

importlib.reload(settings)
# importlib.reload(models)
importlib.reload(helper_funtions)
importlib.reload(preprocess)

from models.LSTM import LSTM_HumanFi, CNN_LSTM, CNN_BiLSTM_TemporalAttention, CNN_BiLSTM_ChannelAttention, CNN_BiLSTM_DualAttention, CNN_BiLSTM_Attention
from models.RadioNet import RadioNet_NeuralWave
from models.ResNet import ECAResNet1D, ECABasicBlock1D, ResNet1D_JARILWWF, OptResNet1D_JARILWWF, OptECAResNet1D_JARILWWF, CustomResNet1D, CustomECAResNet1D
from models.TemporalConvNet import TemporalConvNet

from helper_funtions import grid_search, run_gridsearch_with_seeds
from preprocess import DataPreprocessor
from settings import folder_path_5ghz_10hz_collected, folder_path_60ghz_collected, folder_path_5ghz_200hz_collected, folder_path_60ghz_external, output_path

In [ ]:
input_dim = 52
num_epochs = 500

param_grid = {
    'batch_size': [32],
    'learning_rate': [0.0007], # 0.001,
    'optimizer': ['adam'],
    'mixup_alpha': [0.0, 0.4],
    'smoothing_prob': [0.0, 0.5],
    'model': [],
}

hidden_dims = [64, 128] # 32, 256
num_layers_list = [1, 2]
bidirectional_flags = [False, True]
dropouts = [0.2, 0.5]

for hidden_dim in hidden_dims:
    for num_layers in num_layers_list:
        for bidirectional in bidirectional_flags:
            for dropout in dropouts:
                param_grid['model'].append({
                    'model_class': LSTM_HumanFi,
                    'model_args': {
                        'input_dim': input_dim,
                        'hidden_dim': hidden_dim,
                        'num_layers': num_layers,
                        'bidirectional': bidirectional,
                        'dropout': dropout
                    },
                    'num_epochs': num_epochs
                })

tcn_channel_options = [
    # [32, 64],
    [64, 128],
    [64, 128, 128],
    [64, 128, 256],
    # [128, 256, 512],
]
tcn_kernel_sizes = [2, 3] # 5
tcn_dropout_rates = [0.2, 0.5]

for channels in tcn_channel_options:
    for kernel_size in tcn_kernel_sizes:
        for dropout in tcn_dropout_rates:
            param_grid['model'].append({
                'model_class': TemporalConvNet,
                'model_args': {
                    'num_inputs': input_dim,
                    'num_channels': channels,
                    'kernel_size': kernel_size,
                    'dropout': dropout
                },
                'num_epochs': num_epochs
            })

cnn_lstm_hidden_dims = [64, 128] # 256
cnn_lstm_layers = [1, 2]

for hidden_dim in cnn_lstm_hidden_dims:
    for num_layers in cnn_lstm_layers:
        param_grid['model'].append({
            'model_class': CNN_BiLSTM_Attention,
            'model_args': {
                'input_dim': input_dim,
                'cnn_filters': 64,
                'lstm_units': hidden_dim,
                'num_layers': num_layers,
            },
            'num_epochs': num_epochs
        })

        param_grid['model'].append({
            'model_class': CNN_BiLSTM_TemporalAttention,
            'model_args': {
                'input_dim': input_dim,
                'cnn_channels': 64,
                'lstm_hidden_dim': hidden_dim,
                'lstm_layers': num_layers,
            },
            'num_epochs': num_epochs
        })

        # param_grid['model'].append({
        #     'model_class': CNN_BiLSTM_ChannelAttention,
        #     'model_args': {
        #         'input_dim': input_dim,
        #         'cnn_channels': 64,
        #         'lstm_hidden_dim': hidden_dim,
        #         'lstm_layers': num_layers,
        #     },
        #     'num_epochs': num_epochs
        # })
        #
        # param_grid['model'].append({
        #     'model_class': CNN_BiLSTM_DualAttention,
        #     'model_args': {
        #         'input_dim': input_dim,
        #         'cnn_channels': 64,
        #         'lstm_hidden_dim': hidden_dim,
        #         'lstm_layers': num_layers,
        #     },
        #     'num_epochs': num_epochs
        # })


param_grid['model'] += [
    # {
    #     'model_class': RadioNet_NeuralWave,
    #     'model_args': {'input_dim': input_dim},
    #     'num_epochs': num_epochs,
    #     'data_preprocessor': DataPreprocessor(target_dim=354)
    # },
    
    # the ones commented below are not optimal (proven by early tests)
    # {
    #     'model_class': ECAResNet1D,
    #     'model_args': {'input_channels': input_dim, 'block': ECABasicBlock1D, 'layers': (1, 1, 1, 1)},
    #     'num_epochs': num_epochs
    # },
    # {
    #     'model_class': ECAResNet1D,
    #     'model_args': {'input_channels': input_dim, 'block': ECABasicBlock1D, 'layers': (2, 2, 2, 2)},
    #     'num_epochs': num_epochs
    # },
    # {
    #     'model_class': ResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
    #     'num_epochs': num_epochs,
    # },
    # {
    #     'model_class': ResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    
    
    {
        'model_class': OptResNet1D_JARILWWF,
        'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
        'num_epochs': num_epochs,
    },
    # {
    #     'model_class': OptResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    {
        'model_class': OptECAResNet1D_JARILWWF,
        'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
        'num_epochs': num_epochs,
    },
    # {
    #     'model_class': OptECAResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    {
        'model_class': CustomResNet1D,
        'model_args': {'input_channels': input_dim},
        'num_epochs': num_epochs,
    },
    {
        'model_class': CustomECAResNet1D,
        'model_args': {'input_channels': input_dim},
        'num_epochs': num_epochs,
    },
] 

In [ ]:
gridsearch_results_df = run_gridsearch_with_seeds(
    gridsearch_func=grid_search,
    gridsearch_args={
        'param_grid': param_grid,
        'folder_path': folder_path_5ghz_10hz_collected,
        'background_subtraction': False,
        'seconds_per_sample': 5,
        'rows_per_second': 10,
    },
    n_seeds=3,
    output_dir=output_path,
    filename="gridsearch_5ghz_coll_10hz"
)

In [ ]:
gridsearch_results_df_bgsub = run_gridsearch_with_seeds(
    gridsearch_func=grid_search,
    gridsearch_args={
        'param_grid': param_grid,
        'folder_path': folder_path_5ghz_10hz_collected,
        'background_subtraction': True,
        'seconds_per_sample': 5,
        'rows_per_second': 10,
        'split_signal_stride': 0
    },
    n_seeds=3,
    output_dir=output_path,
    filename="gridsearch_5ghz_coll_10hz"
)

In [ ]:
gridsearch_results_df_bgsub_sps2 = run_gridsearch_with_seeds(
    gridsearch_func=grid_search,
    gridsearch_args={
        'param_grid': param_grid,
        'folder_path': folder_path_5ghz_10hz_collected,
        'background_subtraction': True,
        'seconds_per_sample': 2,
        'rows_per_second': 10,
        'split_signal_stride': 0
    },
    n_seeds=3,
    output_dir=output_path,
    filename="gridsearch_5ghz_coll_10hz"
)